In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# --- Step 1: Load the Pre-trained ELMo model ---

print("Loading ELMo model...")

elmo_model_url = "https://tfhub.dev/google/elmo/3"
elmo_layer = hub.KerasLayer(elmo_model_url, trainable=False, dtype=tf.string, signature='default', signature_outputs_as_dict=True)
print("ELMo model loaded successfully! ✅")

In [3]:
# --- Step 2: Define Sentences with the same word ("bank") in different contexts ---
sentences = [
    "I went to the river bank to fish.", # Context 1: "bank" as in geography
    "I need to go to the bank to withdraw money." # Context 2: "bank" as in finance
]

In [ ]:
# --- Step 3: Get the ELMo Embeddings ---
# The model expects a list of strings.
# The output is a dictionary.
# The shape of the output tensor will be: (number_of_sentences, max_sentence_length, 1024)
elmo_outputs = elmo_layer(tf.constant(sentences))
elmo_embeddings = elmo_outputs['elmo']

print(f"\nShape of the output embeddings tensor: {elmo_embeddings.shape}")

In [5]:
# --- Step 4: Extract the vector for the word "bank" from each sentence ---


# Sentence 1: "I went to the river bank to fish." -> "bank" is the 5th word (index 4)
bank_vec_1 = elmo_embeddings[0, 4, :]

# Sentence 2: "I need to go to the bank to withdraw money." -> "bank" is the 6th word (index 5)
bank_vec_2 = elmo_embeddings[1, 5, :]

# Let's also get the vector for "river" to compare
river_vec = elmo_embeddings[0, 3, :]

In [6]:
# --- Step 5: Compare the vectors using Cosine Similarity ---
# Cosine similarity measures the cosine of the angle between two vectors.
# A value of 1 means they are identical. A value of 0 means they are orthogonal (unrelated).

# Reshape vectors for the function (it expects 2D arrays)
bank_vec_1_reshaped = bank_vec_1.numpy().reshape(1, -1)
bank_vec_2_reshaped = bank_vec_2.numpy().reshape(1, -1)
river_vec_reshaped = river_vec.numpy().reshape(1, -1)

# Compare the two "bank" vectors
similarity_banks = cosine_similarity(bank_vec_1_reshaped, bank_vec_2_reshaped)[0][0]

# Compare "bank" (from sentence 1) with "river"
similarity_bank_river = cosine_similarity(bank_vec_1_reshaped, river_vec_reshaped)[0][0]



In [ ]:

# --- Step 6: Print the results ---
print("\n--- Cosine Similarity Results ---")
print(f"Similarity between 'bank' (river context) and 'bank' (money context): {similarity_banks:.4f}")
print(f"Similarity between 'bank' (river context) and 'river': {similarity_bank_river:.4f}")